In [1]:
""" Learn a policy using DDPG for the reach task"""
import numpy as np
import time
import copy

import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions.multivariate_normal import MultivariateNormal

import gym
import pybullet
import pybulletgym.envs

from collections import deque
from operator import itemgetter
from statistics import mean

import matplotlib.pyplot as plt

np.random.seed(1000)


def weighSync(target_model, source_model, tau=0.001):
    ''' A function to soft update target networks '''
    assert isinstance(tau, float) and tau>0

    for param_target, param_source in zip(target_model.parameters(), source_model.parameters()):
        # Wrap in torch.no_grad() because weights have requires_grad=True, 
        # but we don't need to track this in autograd
        with torch.no_grad():
            param_target = tau*param_source + (1-tau)*param_target
    
    return target_model, source_model


class Replay():
    def __init__(self, buffer_size, init_length, state_dim, action_dim, env):
        """
        A function to initialize the replay buffer.

        param: init_length : Initial number of transitions to collect
        param: state_dim : Size of the state space
        param: action_dim : Size of the action space
        param: env : gym environment object
        """
        assert isinstance(buffer_size, int) and buffer_size>0
        assert isinstance(init_length, int) and init_length>0
        assert isinstance(state_dim, int) and state_dim>0
        assert isinstance(action_dim, int) and action_dim>0
        
        self.buffer_size = buffer_size
        self.state_dim = state_dim
        self.action_dim = action_dim
        
        self.buffer = deque() #list like object for which removing elements from left is faster
        
        s = env.reset()
        for i in range(init_length):
            a = env.action_space.sample()
            s_prime, r, done, _ = env.step(a)
            self.buffer.append({
                's': s,
                'a': a,
                'r': r,
                's_prime': s_prime
            })
    
    def __len__(self):
        ''' Return number of elements in buffer'''
        return len(self.buffer)

    def buffer_add(self, exp):
        """
        A function to add a dictionary to the buffer
        param: exp : A dictionary consisting of state, action, reward , next state and done flag
        """
        assert isinstance(exp, dict) and len(exp) == 4
        assert len(self.buffer) <= self.buffer_size, 'Buffer size exceeded. You fucked up'
        
        if len(self.buffer) < self.buffer_size:
            self.buffer.append(exp)
        else:
            self.buffer.popleft() #removing the 1st element (left most element)
            self.buffer.append(exp)

    def buffer_sample(self, N):
        """
        A function to sample N points from the buffer
        param: N : Number of samples to obtain from the buffer
        """
        assert isinstance(N, int) and N>0
        indices = list(np.random.randint(low=0, high=len(self.buffer), size=N, dtype='int'))
        sample = itemgetter(*indices)(self.buffer) #extarct values at indices from buffer
        
        return sample


class Actor(nn.Module):
    #TODO: Complete the function
    def __init__(self, state_dim, action_dim):
        """
        Initialize the network
        param: state_dim : Size of the state space
        param: action_dim: Size of the action space
        """
        assert isinstance(state_dim, int) and state_dim>0
        assert isinstance(action_dim, int) and action_dim>0
        
        super(Actor, self).__init__()
        self.state_dim = state_dim
        self.action_dim = action_dim
        
        # NN layers and activations
        self.fc1 = nn.Linear(state_dim, 400)
        self.hidden1 = nn.Linear(400, 300)
        self.fc2 = nn.Linear(300, action_dim)
        self.tanh = nn.Tanh()
        self.relu = nn.ReLU()

    def forward(self, state):
        """
        Define the forward pass
        param: state: The state of the environment
        """
#         assert isinstance(state, torch.Tensor)
        assert state.shape == (self.state_dim, ), 'state must be 1D and of size (%d,)'%self.state_dim
        
        if not(isinstance(state, torch.Tensor)):
            state = torch.from_numpy(state).float()
        
        x = state
        x = self.relu(self.fc1(x))
        x = self.relu(self.hidden1(x))
        x = self.tanh(self.fc2(x))

        return x
    
    def getAction(self, state, add_noise_flag = False, noise = 0.1):
        '''
        Returns an action by doing a forward pass. If add_noise_flag is True, 
        action is sampled from a multivariate Normal distributio with stddev = noise and mean = output of net
        
        :rtype: np.ndarray
        '''
#         print('type(state), shape = ',type(state), state.shape)
#         print('self.state_dim = ', self.state_dim)
        assert isinstance(state, np.ndarray)
        assert state.shape == (self.state_dim, )
        assert isinstance(noise, (int, float)) and noise >=0
        assert isinstance(add_noise_flag, bool)
        
        state = torch.from_numpy(state).float()
        action = self.forward(state) #forward pass
        
        if add_noise_flag:
            # Sampling from the nD Gaussian
            m = MultivariateNormal(action, torch.eye(self.action_dim)*noise)
            action = m.sample()
        
        return action.detach().squeeze().numpy()


class Critic(nn.Module):
    def __init__(self, state_dim, action_dim):
        """
        Initialize the critic
        param: state_dim : Size of the state space
        param: action_dim : Size of the action space
        """
        assert isinstance(state_dim, int) and state_dim>0
        assert isinstance(action_dim, int) and action_dim>0
        
        super(Critic, self).__init__()
        self.state_dim = state_dim
        self.action_dim = action_dim
        
        # NN layers and activations
        self.fc1 = nn.Linear(state_dim + action_dim, 400)
        self.hidden1 = nn.Linear(400, 300)
        self.fc2 = nn.Linear(300, 1)
        self.relu = nn.ReLU()

    def forward(self, state, action):
        """ Define the forward pass of the critic """
        assert isinstance(state, np.ndarray)
        assert isinstance(action, np.ndarray)
        assert state.shape == (self.state_dim, ), 'state must be 1D and of size (%d,)'%self.state_dim
        assert action.shape == (self.action_dim, ), 'action must be 1D and of size (%d,)'%self.action_dim
        
        state, action = torch.from_numpy(state).float(), torch.from_numpy(action).float() #numpy to torch tensor
        x = torch.cat((state, action), dim=0) #concatenating to form input
        x = self.relu(self.fc1(x))
        x = self.relu(self.hidden1(x))
        x = self.fc2(x)
        
        return x


class DDPG():
    def __init__(
            self,
            env,
            test_env,
            state_dim,    
            action_dim,
            critic_lr=3e-4,
            actor_lr=3e-4,
            gamma=0.99,
            batch_size=100,
            ev_n_steps=100,
    ):
        """
        Implementing DPPG algorithm from paper - Continuous control with deep reinforcement learning
        link - https://arxiv.org/pdf/1509.02971.pdf
        
        param: env: An gym environment
        param: action_dim: Size of action space
        param: state_dim: Size of state space
        param: critic_lr: Learning rate of the critic
        param: actor_lr: Learning rate of the actor
        param: gamma: The discount factor
        param: batch_size: The batch size for training
        """

        assert isinstance(state_dim, int) and state_dim>0
        assert isinstance(action_dim, int) and action_dim>0
        assert isinstance(batch_size, int) and batch_size>0
        assert isinstance(critic_lr, (int, float)) and critic_lr>0
        assert isinstance(actor_lr, (int, float)) and actor_lr>0
        assert isinstance(gamma, (int, float)) and gamma>0
        assert isinstance(ev_n_steps, (int, float)) and ev_n_steps>0

        self.gamma = gamma
        self.batch_size = batch_size
        self.ev_n_steps = ev_n_steps
        self.env = env
        self.test_env = test_env
        self.num_episodes = 0
        self.avg_rewards = []

        # Create a actor and actor_target with same initial weights
        self.actor = Actor(state_dim, action_dim)
        self.actor = self.init_weights(self.actor) #initialize weights according to paper
        self.actor_target = copy.deepcopy(self.actor) #both networks have the same initial weights 

        # Create a critic and critic_target with same initial weights
        self.critic = Critic(state_dim, action_dim)
        self.critic = self.init_weights(self.critic) #initialize weights according to paper
        self.critic_target = copy.deepcopy(self.critic) #both networks have the same initial weights 

        # Define optimizer for actor and critic
        self.optimizer_actor = optim.Adam(self.actor.parameters(), lr=actor_lr)
        self.optimizer_critic = optim.Adam(self.critic.parameters(), lr=critic_lr)
        self.loss_fn = nn.MSELoss(reduction='sum')

        # Define a replay buffer
        self.ReplayBuffer = Replay(10000, 1000, state_dim, action_dim, self.env)
    
    def init_weights(self, network):
        '''
        Initialize weights (as mentioned in paper) from a uniform distribution 
        based on the fan-in of the layer
        
        WARNING: Will only work if each layer is fully connected
        '''
        
        for name, param in network.named_parameters():
            if 'bias' in name: #if bias param, use f of the same layer
                f = last_f
            else:
                f = param.shape[1] #picking 2nd dim = number of inputs for that layer
                last_f = f
            # Initialize weights by sampling from uniform dist.
            assert isinstance(f, int) and f>0, 'fan in must be int and greater than 0'
            nn.init.uniform_(param.data, a = -1/np.sqrt(f), b = 1/np.sqrt(f))
#             nn.init.uniform_(param.data, a = 0.9999, b = 1)
        
        return network
    
    def save_actor(self, index):
        ''' Saves the policy NN'''
        filename = 'q1_policy_' + str(index) + '.pth.tar'
        state = { 'state_dict': self.actor.state_dict(),
                 'optimizer': self.optimizer_actor.state_dict() }
        torch.save(state, filename)
        
    def save_critic(self, index):
        ''' Saves the critic NN'''
        filename = 'q1_critic_' + str(index) + '.pth.tar'
        state = { 'state_dict': self.critic.state_dict(),
                 'optimizer': self.optimizer_critic.state_dict() }
        torch.save(state, filename)
    
    def update_target_networks(self):
        """
        A function to update the target networks
        """
        weighSync(self.actor_target, self.actor)
        weighSync(self.critic_target, self.critic)

    def update_network(self):
        """
        A function to update the function just once
        """
        pass
    
    def getAverageReward(self):
        ''' Run the policy and return average reward '''
        rewards = []
        s = self.test_env.reset()
        done = False
        while not(done):
            a = self.actor(s).detach().squeeze().numpy()
            s, r, done, _ = self.test_env.step(a)
            rewards.append(r)
        
        avg_reward = sum(rewards)
        assert isinstance(avg_reward, (int, float))
        
        return avg_reward

    def train(self, max_num_steps):
        """
        Train the policy for the given number of iterations
        :param num_steps:The number of steps to train the policy for
        """
        assert isinstance(max_num_steps, int) and max_num_steps>0
        
        gamma = self.gamma
        s = self.env.reset()
        done = False
        for num_steps in range(max_num_steps):
            # Reset env when it reached terminal state
            if done:
                self.num_episodes += 1
                s = self.env.reset()
            
            a = self.actor.getAction(s, add_noise_flag = True, noise = 0.1)
            s_prime, r, done, _ = self.env.step(a)
            
            # Storing transition in buffer
            self.ReplayBuffer.buffer_add({
                's': s,
                'a': a,
                'r': r,
                's_prime': s_prime
            })
            # Sampling N points from buffer
            minibatch = self.ReplayBuffer.buffer_sample(N = self.batch_size)
            
            loss_critic = torch.tensor([0]).float()
            loss_actor = torch.tensor([0]).float()
            
            # Operating on minibatch
            for sample in minibatch:
                s, a, r, s_prime = sample['s'], sample['a'], sample['r'], sample['s_prime']
                # Calculating critic loss
                a_target = self.actor_target.getAction(s_prime, add_noise_flag=False)
                y_i = r + gamma*self.critic_target(s_prime, a_target)
                loss_critic = self.loss_fn(self.critic(s,a), y_i) #mse loss
                
                # Calculating actor loss
                a = self.actor.getAction(s_prime, add_noise_flag=False)
                loss_actor += self.critic(s, a)
            
            # zero gradients of optimizer
            self.optimizer_critic.zero_grad()    
            self.optimizer_actor.zero_grad()
            # Update critic
            loss_critic.backward()
            self.optimizer_critic.step()
            # Update actor
            loss_actor /= -self.batch_size #multiplying with negative so it does gradient ascent
            loss_actor.backward()
            self.optimizer_actor.step()
            # Update target networks
            self.update_target_networks()
            
            if num_steps%100 == 0:
                r = self.getAverageReward()
                self.avg_rewards.append(r)
                print('Num steps: {0} \t Avg Reward: {1:.3f} \t Obj(Actor): {2:.3f} \t Loss(Critic): {3:.3f}'\
                      .format(num_steps, r, loss_actor.item(), loss_critic.item()))
                
            

In [2]:
if __name__ == "__main__":
    # Define the environment
    env = gym.make("modified_gym_env:ReacherPyBulletEnv-v1", rand_init=True)
    test_env = copy.deepcopy(env)
    
    # Define Deep Deterministic Policy Gradient object
    ddpg_object = DDPG(
        env,
        test_env,
        8,
        2,
        critic_lr=1e-3,
        actor_lr=1e-3,
        gamma=0.99,
        batch_size=100,
        ev_n_steps=100, #evaluate every n steps
    )
    # Train the policy
    ddpg_object.train(10000)

#     # Evaluate the final policy
#     state = env.reset()
#     done = False
#     while not done:
#         action = ddpg_object.actor(state).detach().squeeze().numpy()
#         next_state, r, done, _ = env.step(action)
#         env.render()
#         time.sleep(0.1)
#         state = next_state


current_dir=/Users/adnanshahpurwala/anaconda3/envs/ece276c_venv/lib/python3.6/site-packages/pybullet_envs/bullet
options= 
options= 
Num steps: 0 	 Avg Reward: -19.619 	 Obj(Actor): -0.073 	 Loss(Critic): 0.000
Num steps: 50 	 Avg Reward: -17.554 	 Obj(Actor): 0.158 	 Loss(Critic): 0.133
Num steps: 100 	 Avg Reward: -20.268 	 Obj(Actor): -0.145 	 Loss(Critic): 0.126
Num steps: 150 	 Avg Reward: -19.361 	 Obj(Actor): 0.051 	 Loss(Critic): 0.004
Num steps: 200 	 Avg Reward: -18.494 	 Obj(Actor): 0.037 	 Loss(Critic): 0.040
Num steps: 250 	 Avg Reward: -20.747 	 Obj(Actor): 0.027 	 Loss(Critic): 0.002
Num steps: 300 	 Avg Reward: -22.096 	 Obj(Actor): -0.024 	 Loss(Critic): 0.039
Num steps: 350 	 Avg Reward: -21.973 	 Obj(Actor): 0.044 	 Loss(Critic): 0.125
Num steps: 400 	 Avg Reward: -19.606 	 Obj(Actor): 0.009 	 Loss(Critic): 0.004
Num steps: 450 	 Avg Reward: -18.405 	 Obj(Actor): -0.009 	 Loss(Critic): 0.020
Num steps: 500 	 Avg Reward: -21.846 	 Obj(Actor): 0.026 	 Loss(Critic): 0.0

In [ ]:
# ddpg_object.save_actor(2)